<img src="DAMASK_banner.png">

# <font color=blue> Importing Necessary Modules </font>

In [ ]:
import numpy as np
import damask
from damask import ConfigMaterial as cm

***
<br><br>
# <center><font color=blue> **SECTION - 01:** Cleaning .ang files </font><center>

- ### <font color=blue>That is changing values outside the range of [0, 2π] to be inside</font>
- ### <font color=scarlet> Assumes that euler angles in the ang file are in radians. </font>

<font color=red>NOTE: This needs to be done, because .ang files as created may contain some values greater than [0, 2π], which while will be processed by DREAM3D, but during creation of material file, damask will raise error.</font>

In [ ]:
GENERATED_ANG_FILE = "YOUR_CREATED_ANG_FILE_FILEPATH"
# Obtained ang file path and filename, from tsl or ebsd or elsewhere
CLEANED_ANG_FILE = "YOUR_OUTPUT_FILE_FILEPATH"
# What you want cleaned ang files name to be and where you need it placed
HEADER_LINES = "HEADER_LINES_IN_ANG_FILE"
# Lines starting with # (i.e. before data of ebsd starts)

### <font color=grey>Creating few helper functions</font>

In [1]:
def outside_range(euler_angle: str, range_min: float = 0, range_max: float = 2 * np.pi) -> bool:
    """Takes in euler angle value (as a string), outputs whether they are outside the accepted range of [0, 2pi],
        Returns True if angle is outside the accepted range,
        False otherwise."""
    angle_radians = float(euler_angle)
    return not (range_min <= angle_radians < range_max)

def convert_in_range(euler_angle: str, range_min: float = 0, range_max: float = 2 * np.pi) -> str:
    """Takes in euler angle (as a string), outputs a value by subtracting or adding 2pi values a i in range value."""
    while outside_range(euler_angle, range_min, range_max):
        angle_radians = float(euler_angle)
        if float(euler_angle) < range_min:
            euler_angle = f"{angle_radians + range_max:.5f}"
        else:
            euler_angle = f"{angle_radians - range_max:.5f}"
    return euler_angle

NameError: name 'np' is not defined

### <font color=red>Taking in data from GENERATE_ANG_FILE and cleaning it </font>

In [ ]:
header = []
data_lines = []
with open(GENERATED_ANG_FILE, "r") as file_read:
    all_data_lines = file_read.readlines()
    for header_lines in all_data_lines[:HEADER_LINES]:
        # Taking in all header lines in the header list.
        header.append(header_lines)
        
    for data_line in all_data_lines[HEADER_LINES:]:
        # Going through all data_line, first convert data_line in a list
        data_line = data_line.split()
        data_line[0] = convert_in_range(data_line[0])
        data_line[1] = convert_in_range(data_line[1], 0, np.pi)
        data_line[2] = convert_in_range(data_line[2])
        data_lines.append(data_line)

### <font color=green>Exporting clean data as CLEANED_ANG_FILE</font>

In [ ]:
# Exporting data 
with open(CLEANED_ANG_FILE, "w") as file_write:
    for header_line in header:
        file_write.write(header_line)
    for datas in data_lines:
        line = ""
        for data in datas:
            line += data
            line += "\t"
        file_write.write(line + "\n")

## <font color=blue>Converting CLEANED_ANG_FILE to dream3d file</font>

### To convert .ang type files to dream3d file, you can use following DREAM3D pipeline
- #### <a href="DREAM3D_Pipelines/AngTodream3d.json">Pipeline to convert .ang file to dream3d file<a> <br>
    This is placed in DREAM3D_Pipelines folder with filename, "AngTodream3d.json"

### <font color=red> Sometime DREAM3D may throw following error when using above pipeline</font>
#### &emsp; &emsp;<font color=red>import EDAX EBSD data(.ang) </font> &emsp; &emsp; END of ang file reached before all data was parsed.
### <font color=green> Solution: <br> &emsp; &emsp; In this case Go to NROWS line of the header, and decrease the count by 1, that should resolve the issue.</font>

# <font color=blue>SECTION - 01_2 : Converting ctf files to dream3d files</font>

### Converting to dream3d file is quite easy, cleaning and others are not required, the pipeline to use is:
- #### <a href="DREAM3D_Pipelines/ctf_to_dream3d.json">Pipeline to convert .ctf file to .dream3d file</a> <br>
This is placed in DREAM3D_Pieplines folder with filename "ctf_to_dream3d.json"

***
# <font color=scarlet><center>SECTION - 01 : END </center></font>
***

<br> <br> <br>

# <font color=blue><center> SECTION-02: Configuring simulation from dream3d file </center></font> <br>
- ### <font color=blue>Creating a material.yaml file using dream3d file and available material data, (collected from literature or caluclated)</font>
- ### <font color=blue>Creating a gridd file(.vti) file using dream3d file </font>

## <center><font color=purple> SECTION-02.1 Creating material file for single phase material </font><center>

In [4]:
DREAM_3D_FILE = "YOUR_DREAM3D_FILEPATH"
# Path to dream3d file created from methods mentioned above
OUTPUT_GRID_FILE = "GRID_FILE_NAME"
# What you want grid file name to be
MATERIAL_CONTANTS_FILE = "MATERIAL_CONSTANTS_FILENAME"
# yaml file where you have saved material data needed for simulation, examples in damask GitHub pages

In [ ]:
# creating a config_material object from dream 3d file
config_material = cm.load_DREAM3D(DREAM_3D_FILE)

# Defining homogenization as type pass, if you are using ebsd data from SEM it should be type pass only
config_material['homogenization']['direct']['mechanical'] = {'type': 'pass'}